<a href="https://colab.research.google.com/github/michaelmoschitto/CoverLetterGPT/blob/main/HFCustomDatasetEx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=e4c408a50f3f62aa211a57057cb120345dd02f087270e4247832dc267d906c62
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2023-05-31 18:55:50--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  21.3MB/s    in 5.6s    

2023-05-31 18:55:56 (14.4 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir == "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')
# display(train_texts)

In [4]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [5]:
train_texts = train_texts[:50]
val_texts = val_texts[:50]
test_texts = test_texts[:50]

train_labels = train_labels[:50]
val_labels = val_labels[:50]
test_labels = test_labels[:50]

In [6]:
print(type(train_texts), type(train_labels))
print(len(train_texts), len(train_labels))
print(train_labels[0], train_texts[0][:])

<class 'list'> <class 'list'>
50 50
1 *May contain spoilers* *May contain spoilers*<br /><br />In the age of Shrek(the movie) & Pixar(the studio), this is a much more traditional animation film. It put together some characters that normally wouldn't be seen together(not to mentioned, try to save a human baby and bring it back to his father). They begin as enemies and end as best friend. If this sound like a Disney film, it is(only made through 20th Century Fox). The trailer to the movie was one of the best I've seen in ages, but the movie doesn't live to the expectation the trailer set. The problem lie with the fact that the makers of the film didn't made up their mind who is the target audience of the film. Yes, there are some jokes in the film that only adults will understand but the film is mostly aimed at children. The parents will enjoy the fact that for 90 min. their children's attention is focused on something else than them. The backgrounds are excellent and the voice are good 

In [7]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [8]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [10]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [11]:
train_dataset[5]

{'input_ids': tensor([  101,  2023,  4746, 12883,  2033,   999,  2045,  2003,  2053, 17211,
          2000,  2009,  1998,  2003,  2521,  2205,  3809,  2000,  2022,  2170,
          1000,  4569,  1000,   999,  2009,  1005,  1055,  2074,  2521,  2205,
          4547,  2005,  2026, 16663,   999,  1996,  3494,  2024,  2200, 12991,
         13874,  2094,  1998, 14477, 21512, 21682,  1012,  1996, 14811,  2024,
         21707,  1998,  1996, 25288,  2024,  2074,  5567,  2058,  1998,  2058,
          2153,  1012,  2073,  1005,  1055,  1996,  4569,  1999,  2009,  1029,
          2036,  1045,  2514,  2023,  2038,  2042,  2006,  1996,  4035,  2005,
          2521,  2205,  2146,  1998,  2003,  3743,  2126,  2205,  2172,  1012,
          2515,  2009,  2428,  2342,  2000,  2031,  1037, 10453,  2006,  1056,
          1012,  1058,  2296,  1016,  2030,  1017,  2706,  2043,  1037,  4435,
          2047,  2265,  3216,  2041,  1997,  4178,  1029,  1045,  2228,  2009,
          1005,  1055,  2051,  2008,  1

In [16]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [25]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    f"distilledbert-finetuned-sentiment",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=10,  # batch size per device during training
    per_device_eval_batch_size=5,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    push_to_hub=True,
    # metric_for_best_model="accuracy",
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    # compute_metrics=compute_metrics
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

Epoch,Training Loss,Validation Loss
1,No log,0.693738
2,0.689200,0.693119
3,0.689200,0.692022
4,0.691100,0.690662
5,0.691100,0.689028


TrainOutput(global_step=25, training_loss=0.6893087196350097, metrics={'train_runtime': 647.603, 'train_samples_per_second': 0.386, 'train_steps_per_second': 0.039, 'total_flos': 33116849664000.0, 'train_loss': 0.6893087196350097, 'epoch': 5.0})

In [23]:
trainer.evaluate()

{'eval_loss': 0.7034949660301208,
 'eval_runtime': 25.1925,
 'eval_samples_per_second': 1.985,
 'eval_steps_per_second': 0.04,
 'epoch': 3.0}

In [24]:
from transformers import DistilBertTokenizer

# Load the trained model and tokenizer
model_path = "./results"  # Path to the trained model directory
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilledbert-finetuned-sentiment")

# Example texts for testing
examples = [
    "This is a positive sentence.",
    "I'm feeling neutral about this.",
    "Negative feedback on the product.",
]

# Tokenize and encode the examples
encoded_inputs = tokenizer(examples, padding=True, truncation=True, return_tensors="pt")

# Make predictions
with torch.no_grad():
    model.eval()
    outputs = model(**encoded_inputs)

# # Get the predicted labels
predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()

# # Map the predicted labels to their corresponding text
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
predicted_texts = [label_map[label] for label in predicted_labels]

# # Print the predicted labels for the examples
for text, label in zip(examples, predicted_texts):
    print(f"Text: {text}\nPredicted Label: {label}\n")


Text: This is a positive sentence.
Predicted Label: Neutral

Text: I'm feeling neutral about this.
Predicted Label: Neutral

Text: Negative feedback on the product.
Predicted Label: Neutral

